# Dự đoán cảm xúc từ các bài bình luận sản phẩm


Mục tiêu của notebook đầu tiên này là khám phá hồi quy Logistic và kỹ thuật đặc trưng với các hàm sklearn hiện có.

Trong notebook này, chúng ta sẽ sử dụng dữ liệu bình luận sản phẩm từ Amazon.com để dự đoán cảm xúc về một sản phẩm (từ bình luận) là tích cực hay tiêu cực.

* Sử dụng pandas để thực hiện một số kỹ thuật đặc trưng.
* Huấn luyện mô hình hồi quy Logistic để dự đoán cảm xúc của các bình luận về sản phẩm.
* Kiểm tra các trọng số (hệ số) của mô hình hồi quy Logistic đã huấn luyện.
* Đưa ra dự đoán (cả lớp và xác suất) của cảm xúc cho một bình luận sản phẩm mới.
* Viết hàm tính **accuracy** của mô hình biết trọng số hồi quy, các yếu tố dự đoán và nhãn ground truth.
* Kiểm tra hệ số của mô hình hồi quy Logistic và giải thích ý nghĩa của chúng.
* So sánh nhiều mô hình hồi quy Logistic.

Hãy bắt đầu thôi!

In [25]:
# Import một số thư viện

import pandas
import numpy as np
from sklearn.model_selection import train_test_split

In [26]:
# Import dữ liệu amazon_baby.csv vào pandas dataframe
products_df = pandas.read_csv('../dataset/amazon_baby.csv')
products_df.head()

,reviewerID,asin,reviewerName,helpful,helpful_num,helpful_den,reviewText,overall,summary,unixReviewTime,reviewTime,exclamationcount,questioncount,charcount,wordcount,capcount,avgrating,diffrating,ishelpful
0,A3NMPMELAZC8ZY,097293751X,Jakell,"[3, 3]",3,3,This book is perfect! I'm a first time new mo...,5,Great for newborns,1359244800,"01 27, 2013",1,0,250,46,0,4.0,1.0,1
1,A3O4ATU0ENBKTU,097293751X,MAPN,"[1, 1]",1,1,I use this so that our babysitter (grandma) ca...,5,Compact and Easy way to record the milestones,1361836800,"02 26, 2013",0,0,734,148,0,4.0,1.0,1
2,A2SYNL4YX73KNY,097293751X,"R. Davidson ""Jrdpa""","[2, 2]",2,2,"I like this log, but think it would work bette...",3,Needs clearer AM & PM,1369008000,"05 20, 2013",0,0,288,59,2,4.0,1.0,1
3,A2Q2A6JKY95RTP,097293751X,R. Garrelts,"[2, 2]",2,2,My wife and I have a six month old baby boy an...,3,Expensive and Somewhat Limited Format,1381968000,"10 17, 2013",0,0,2959,505,9,4.0,1.0,1
4,A21I33AWNOWMK8,9729375011,EmilyS,"[1, 2]",1,2,I have used this book since my son was born. ...,5,Great product!,1364256000,"03 26, 2013",0,0,595,117,0,4.5,0.5,0


In [5]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56950 entries, 0 to 56949
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   reviewerID        56950 non-null  object 
 1   asin              56950 non-null  object 
 2   reviewerName      56768 non-null  object 
 3   helpful           56950 non-null  object 
 4   helpful_num       56950 non-null  int64  
 5   helpful_den       56950 non-null  int64  
 6   reviewText        56950 non-null  object 
 7   overall           56950 non-null  int64  
 8   summary           56950 non-null  object 
 9   unixReviewTime    56950 non-null  int64  
 10  reviewTime        56950 non-null  object 
 11  exclamationcount  56950 non-null  int64  
 12  questioncount     56950 non-null  int64  
 13  charcount         56950 non-null  int64  
 14  wordcount         56950 non-null  int64  
 15  capcount          56950 non-null  int64  
 16  avgrating         56950 non-null  float6

### 1. Tiền xử lý dữ liệu
Trước khi sử dụng dữ liệu đánh giá, chúng ta cần thực hiện tiền xử lý
<br>
Trước tiên, loại bỏ dấu câu khỏi các từ
<br>
Ví dụ: **good!** --> **good**, **resturant.** --> **resturant**

In [8]:
def remove_punctuation(text):
    import string
    return text.translate(string.punctuation)

products_df = products_df.fillna({'reviewText':''})  # điền vào các N/A ở cột review
products_df['review_clean'] = products_df['reviewText'].apply(remove_punctuation)

Chúng ta sẽ bỏ qua tất cả các bình luận có rating = 3 vì chúng có cảm xúc trung lập.

In [9]:
products_df = products_df[products_df['overall'] != 3]

Bây giờ, chúng ta sẽ chỉ định các bình luận có rating từ 4 trở lên là bình luận tích cực, trong khi các bình luận từ 2 trở xuống là tiêu cực. Đối với cột sentiment, chúng ta sử dụng +1 cho nhãn positive class (lớp tích cực) và -1 cho nhãn negative class (lớp tiêu cực). Chúng ta nên tạo một hàm ẩn danh chuyển đổi rating thành class label, sau đó áp dụng hàm đó cho mọi phần tử trong cột rating. Trong Pandas, chúng ta sẽ sử dụng appl ():

In [10]:
products_df['sentiment'] = products_df['overall'].apply(lambda rating : +1 if rating > 3 else -1)

Hãy thực hiện phân tách train/test với 80% dữ liệu trong tập huấn luyện và 20% dữ liệu trong tập kiểm tra.
Chúng ta sẽ thiết lập seed để đảm bảo nhận được các kết quả tương tự.

In [11]:
np.random.seed(1)

train_data, test_data = train_test_split(products_df, test_size=0.2)

### 2. Biểu diễn đặc trưng
 Bây giờ, chúng ta sẽ tính số từ cho mỗi từ xuất hiện trong các bình luận. Vectơ đếm từ thường được gọi là đặc trưng túi từ. Vì hầu hết các từ chỉ xuất hiện trong một số bình luận nên các vectơ đếm từ khá thưa thớt. Do đó, scikit-learn và nhiều công cụ khác sử dụng ma trận thưa thớt để lưu trữ tập hợp các vectơ đếm từ.
<br>
Có thể tham khảo [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) từ tài liệu scikit.

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# Sử dụng mẫu token này để giữ các từ có một chữ cái
# Trước tiên, tìm hiểu từ vựng từ dữ liệu huấn luyện và gán cột cho từ
# Sau đó chuyển đổi dữ liệu huấn luyện thành ma trận thưa thớt
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Tiếp theo, chuyển đổi dữ liệu kiểm tra thành ma trận thưa thớt, sử dụng cùng một ánh xạ từ-cột
test_matrix = vectorizer.transform(test_data['review_clean'])

train_matrix.shape, test_matrix.shape

((39589, 39971), (9898, 39971))

### 3. Huấn luyện mô hình hồi quy logistic

Tìm hiểu phân loại hồi quy logistic bằng cách sử dụng dữ liệu huấn luyện. Chúng ta đang sử dụng scikit-learn, nên hãy tạo một instance của [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), sau đó gọi phương thức fit() để huấn luyện phân loại. Mô hình này nên sử dụng ma trận đếm từ thưa thớt (**train_matrix**) làm đặc trưng và cột **sentiment** của **train_data** làm mục tiêu. Sử dụng các giá trị mặc định cho các tham số khác.

In [14]:
### VIẾT CODE Ở ĐÂY ###
from sklearn.linear_model import LogisticRegression

X, y = train_matrix, train_data['sentiment'].values

clf = LogisticRegression().fit(X, y)
### VIẾT CODE Ở ĐÂY ###

/home/funny/miniconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Kiểm tra train accuracy.
<br>
Nên thu được train accuracy là: 0.9474291796913067

In [15]:
print ('Train accuracy: {}'.format(clf.score(X, y)))

Train accuracy: 0.9452373133951351


Kiểm tra test accuracy. 
Nên thu được test accuracy là: 0.9345746754220263

In [16]:
print ('Test accuracy: {}'.format(clf.score(test_matrix, test_data['sentiment'])))

Test accuracy: 0.8994746413416852


**Quiz**: Có bao nhiêu trọng số >= 0?
<br>
**Đáp án**:
<br>
`sum(1 for c in clf.coef_[0] if c >= 0)`

### 4. Đưa ra dự đoán

Trước tiên, hãy tính score cho 6 mẫu từ **test_data**.
<br>
Nhớ lại từ bài giảng: $$\mbox{score}_i = \mathbf{w}^\intercal h(\mathbf{x}_i)$$

In [18]:
sample_test_data = test_data[105:110]
print(sample_test_data['review_clean'])
print(sample_test_data['overall'])
print(sample_test_data['sentiment'])

35366    We received this as a Christmas gift and frank...
38513    I wouldn't go with any other brand! I love my ...
608      These bottles are superior to any on the marke...
1841     In the middle of the night when the baby is sc...
52128    I used a summer infant monitor with my first c...
Name: review_clean, dtype: object
35366    5
38513    5
608      5
1841     5
52128    5
Name: overall, dtype: int64
35366    1
38513    1
608      1
1841     1
52128    1
Name: sentiment, dtype: int64


In [19]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = clf.decision_function(sample_test_matrix)
print (scores)

[ 7.17833028 15.15345031 -5.00869776  1.22749842 10.67968595]


Các score này được sử dụng để dự đoán lớp như sau:
<br>
$$\hat{y}_i = \begin{cases} +1 & \text{if } \mathbf{w}^\intercal h(\mathbf{x}_i) > 0\\ -1 &\text{if } \mathbf{w}^\intercal h(\mathbf{x}_i) \leq 0 \end{cases}$$

Chúng ta có thể sử dụng hàm predict() từ scikit-learn.

In [20]:
print (clf.predict(sample_test_matrix))

[ 1  1 -1  1  1]


 Nhớ lại từ bài giảng, chúng ta cũng có thể tính xác suất dự đoán từ score sử dụng:
 <br>
 $$P(y_i = +1 | \mathbf{x}_i, \mathbf{w}) = \dfrac{1}{1+\exp{(-\mathbf{w}^\intercal h(\mathbf{x}_i))}}$$

Chúng ta có thể sử dụng hàm predict_proba() từ scikit-learn.

In [21]:
print (clf.predict_proba(sample_test_matrix))
# LƯU Ý: P(yi=+1|xi,w) thể hiện trong cột thứ hai, cột thứ nhất thể hiện P(yi=-1|xi,w)

[[7.62359041e-04 9.99237641e-01]
 [2.62385624e-07 9.99999738e-01]
 [9.93364725e-01 6.63527515e-03]
 [2.26619560e-01 7.73380440e-01]
 [2.30070709e-05 9.99976993e-01]]


Tính accuracy:
<br>
$$\mbox{accuracy} = \dfrac{\mbox{# correctly classified examples}}{\mbox{# total examples}}$$
<br>
Chúng ta có thể sử dụng hàm score() từ scikit-learn.

In [22]:
# Accuracy của 6 mẫu trên :DD
print (clf.score(sample_test_matrix, sample_test_data['sentiment']))

0.8


In [23]:
# Accuracy cho toàn bộ test_data
print ('Test accuracy: {}'.format(clf.score(test_matrix, test_data['sentiment'])))

Test accuracy: 0.8994746413416852
